This dataset is used by Just Eat Takeaway in food delivery company in Europe to recruite data analytics professional.

In 3 hours, the candidate needs to finish the following challenge.

Imagine you need to present your insights and solutions to the Meituan logistics management team who is data savvy and keen to understand and solve the following business questions.
Please prepare the slides so that the presentation can be completed within an hour with questions:


*   What are the root causes of PtoD (duration from order placement to delivery to customer) and how to improve?
*   Is there any pattern for courier rejection (is_waybill_grabbed)? If there is a significant pattern, how can you improve the order-courier assignment algorithm in order to mitigate the impacts of courier rejection.



### Link to the Github



In [ ]:
!git clone 'https://github.com/meituan/Meituan-INFORMS-TSL-Research-Challenge'
import os
os.chdir('Meituan-INFORMS-TSL-Research-Challenge')

Cloning into 'Meituan-INFORMS-TSL-Research-Challenge'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 45 (delta 7), reused 2 (delta 2), pack-reused 33 (from 1)
Receiving objects: 100% (45/45), 79.05 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (17/17), done.


### Load the datasets

In [ ]:
import pandas as pd
import zipfile
import lightgbm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
# Specify the path to the zip file
zip_path = "all_waybill_info_meituan_0322.csv.zip"

# Open the zip file and read the CSV within it
with zipfile.ZipFile(zip_path) as z:
    # List the files in the zip archive to find the CSV file name
    csv_filename = z.namelist()[0]  # This assumes the CSV is the first file in the zip
    # Read the CSV file directly from the zip
    df_info = pd.read_csv(z.open(csv_filename))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
df_wave = pd.read_csv('courier_wave_info_meituan.csv')
df_rider = pd.read_csv('dispatch_rider_meituan.csv')
df_waybill = pd.read_csv('dispatch_waybill_meituan.csv')

### Data Wrangling and Feature Engineering

#### Convert the unix time to China Time

In [ ]:
def convert_time_col(df_info):
  for col in df_info.columns:
      if 'time' in col:
        df_info[col] =df_info[col].replace(0,'')
        df_info[col] = pd.to_datetime(df_info[col], unit='s', utc=True).dt.tz_convert('Asia/Shanghai')
        # Remove timezone information to get local time in Beijing without '+08:00'
        df_info[col] = df_info[col].dt.tz_localize(None)
  return df_info

df_info = convert_time_col(df_info)
df_wave = convert_time_col(df_wave)
df_rider = convert_time_col(df_rider)
df_waybill = convert_time_col(df_waybill)

In [ ]:
df_wave['courier_wave'] = df_wave['dt'].astype(str)+'_'+ df_wave['courier_id'].astype(str)+'_'+df_wave['wave_id'].astype(str)
df_wave['order_ids'] = df_wave['order_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_wave['order_count'] = df_wave['order_ids'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df_wave_exp = df_wave.explode('order_ids').reset_index(drop=True).rename(columns={'order_ids':'order_id'})

#### create additional features related to time

In [ ]:
# prompt: for all the columns with column name ending with '_time' and having pd.datetime as datatype,  create column with '_hour', '_date' but removing '_time'

def create_time_features(df):
  for column in df.columns:
    if column.endswith('_time') and pd.api.types.is_datetime64_any_dtype(df[column]):
      df[column.replace('_time', '_hour')] = df[column].dt.hour
      df[column.replace('_time', '_date')] = df[column].dt.date
  return df


# Example Usage:
# Assuming df_info is your DataFrame
df_info = create_time_features(df_info)

### create additional features related to time segments

In [ ]:
def cate_time(row):
    hour = row.hour
    if hour <= 10:
        return '0.pre-lunch'
    elif hour in [11, 12]:
        return '1.lunch'
    elif 13 <= hour <= 16:
        return '2.pre-dinner'
    elif 17 <= hour <= 19:
        return '3.dinner'
    else:
        return '4.after dinner'

def cate_time_2(row):
    hour = row.hour
    if hour in [7,9]:
        return '0.morning rush'
    elif hour in [11,12]:
        return '1.lunch peak'
    elif hour in [17,18,19]:
        return '2.dinner peak'
    else:
        return 'non-peak'

In [ ]:
def apply_time_categorization(df, categorize_func_list):
  for col in df.columns:
    if col.endswith('_time') and pd.api.types.is_datetime64_any_dtype(df[col]):
      for categorize_func in categorize_func_list:
        new_col_name = col.replace('_time', f'_{categorize_func.__name__}')
        df[new_col_name] = df[col].apply(categorize_func)
  return df

df_info_ok = df_info.copy()  # Create a copy to avoid modifying the original DataFrame
categorize_func_list = [cate_time, cate_time_2]
df_info_ok = apply_time_categorization(df_info_ok, categorize_func_list)

### BQ1: PtoD

In [ ]:
df_info_delivered = df_info_ok[df_info_ok['is_courier_grabbed']==1]

In [ ]:
df_info_delivered['push_min'] = (df_info_delivered['order_push_time'] - df_info_delivered['platform_order_time']).dt.total_seconds() / 60
df_info_delivered['first_rider_min'] = (df_info_delivered['dispatch_time'] - df_info_delivered['order_push_time']).dt.total_seconds() / 60
df_info_delivered['rider_accept_min'] = (df_info_delivered['grab_time'] - df_info_delivered['dispatch_time']).dt.total_seconds() / 60
df_info_delivered['meal_prep_and_wait_min'] = (df_info_delivered['fetch_time'] - df_info_delivered['grab_time']).dt.total_seconds() / 60
df_info_delivered['meal_prep_min'] = (df_info_delivered['estimate_meal_prepare_time'] - df_info_delivered['grab_time']).dt.total_seconds() / 60
df_info_delivered['shop_wait_min'] = (df_info_delivered['fetch_time'] - df_info_delivered['estimate_meal_prepare_time']).dt.total_seconds() / 60
df_info_delivered['delivery_min'] = (df_info_delivered['arrive_time'] - df_info_delivered['fetch_time']).dt.total_seconds() / 60

df_info_delivered['estimated_tot_travel_min'] = (df_info_delivered['estimate_arrived_time'] - df_info_delivered['grab_time']).dt.total_seconds() / 60
df_info_delivered['actual_duration_min'] = (df_info_delivered['arrive_time'] - df_info_delivered['dispatch_time']).dt.total_seconds() / 60

<ipython-input-10-51bc6214ab60>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info_delivered['push_min'] = (df_info_delivered['order_push_time'] - df_info_delivered['platform_order_time']).dt.total_seconds() / 60
<ipython-input-10-51bc6214ab60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info_delivered['first_rider_min'] = (df_info_delivered['dispatch_time'] - df_info_delivered['order_push_time']).dt.total_seconds() / 60
<ipython-input-10-51bc6214ab60>:3: SettingWithCopyWarning: 
A value is tr

In [ ]:
df_info_delivered['is_ontime'] = df_info_delivered['estimate_arrived_time']>df_info_delivered['arrive_time']

<ipython-input-11-7e9e76e1a010>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info_delivered['is_ontime'] = df_info_delivered['estimate_arrived_time']>df_info_delivered['arrive_time']


#### Compare the on-time delivered vs delayed order re. time spent on each step to find where the gap is.



In [ ]:
df_info_delivered_clean = df_info_delivered[(df_info_delivered['push_min']>0) & (df_info_delivered['rider_accept_min']>0) & (df_info_delivered['first_rider_min']>0)\
    & (df_info_delivered['meal_prep_and_wait_min']>0) \
        # & (df_info_delivered['meal_prep_min']>0) & (df_info_ok['shop_wait_min']>0) \
        & (df_info_delivered['delivery_min']>0) \
          #& (df_info_ok['is_prebook']==0)]
]


In [ ]:
df_info_delivered_clean_adhoc = df_info_delivered_clean[df_info_delivered_clean['is_prebook']==0]

In [ ]:
df_info_delivered_clean_prebook = df_info_delivered_clean[df_info_delivered_clean['is_prebook']==1]

In [ ]:
def cal_step_p2d_diff(df):
  return df.groupby('is_ontime')[['push_min','first_rider_min','rider_accept_min','meal_prep_and_wait_min',\
                                              'delivery_min']].mean().reset_index()

##### Prebook orders

In [ ]:
cal_step_p2d_diff(df_info_delivered_clean_prebook)

,is_ontime,push_min,first_rider_min,rider_accept_min,meal_prep_and_wait_min,delivery_min
0,False,13.725275,105.238426,0.72427,21.524842,27.456123
1,True,11.840138,110.621152,0.52678,14.509440,22.450657


##### Adhoc orders

In [ ]:
cal_step_p2d_diff(df_info_delivered_clean_adhoc)

,is_ontime,push_min,first_rider_min,rider_accept_min,meal_prep_and_wait_min,delivery_min
0,False,1.581125,4.364680,0.715300,13.962434,23.652268
1,True,0.858577,2.888451,0.379754,9.319638,13.555214


##### no distinguishment

In [ ]:
cal_step_p2d_diff(df_info_delivered_clean)

,is_ontime,push_min,first_rider_min,rider_accept_min,meal_prep_and_wait_min,delivery_min
0,False,2.159244,9.166750,0.715727,14.322440,23.833349
1,True,1.230717,6.539265,0.384737,9.495509,13.856660


The main gap is coming from waiting time to pick up the order, and the delivery time.


### Factors that might lead to the delay of the last two steps.


*   Traffic
*   Workload intensity of rider
*   Workload intensity of restaurant
*   Distance for delivery
*   Rider/Restaurant capability

### The first 3 are time-relevant, the last 2 are time-invariant.



#### How on-time rate, active riders, order placed change over 24 hours

In [ ]:
df_info_delivered_clean_adhoc.groupby('grab_hour')\
.agg(avg_on_time_rate = ('is_ontime','mean'),
     num_orders = ('order_id','nunique')).reset_index()

,grab_hour,avg_on_time_rate,num_orders
0,0.0,0.882012,5009
1,1.0,0.889271,3197
2,2.0,0.846154,1989
3,3.0,0.882904,1281
4,4.0,0.886414,898
5,5.0,0.903226,837
6,6.0,0.876611,2561
7,7.0,0.902375,5726
8,8.0,0.890366,9404
9,9.0,0.912455,11000


In [ ]:
df_info.groupby(['grab_date','grab_hour'])\
.agg(num_riders = ('courier_id','nunique')).reset_index()\
.groupby('grab_hour').agg(daily_avg_rider = ('num_riders','mean'))

,daily_avg_rider
grab_hour,
0.0,210.888889
1.0,156.625000
2.0,95.625000
3.0,66.625000
4.0,52.750000
5.0,69.375000
6.0,172.875000
7.0,356.375000
8.0,514.625000


### Hypothesis Testing

#### meal prep takes longer due to spike in demand

In [ ]:
df_info_delivered_clean_adhoc[df_info_delivered_clean_adhoc['meal_prep_min']>0].groupby('grab_hour')\
 [['meal_prep_min']].mean()

,meal_prep_min
grab_hour,
0.0,10.105053
1.0,10.578481
2.0,10.198831
3.0,10.094451
4.0,10.407722
5.0,10.990394
6.0,10.395204
7.0,10.875186
8.0,10.485270


In [ ]:
df_info_delivered_clean[df_info_delivered_clean['meal_prep_and_wait_min']>0]\
.groupby(['is_prebook','grab_hour'])[['meal_prep_and_wait_min']].mean().reset_index()

,is_prebook,grab_hour,meal_prep_and_wait_min
0,0,0.0,8.901767
1,0,1.0,9.233057
2,0,2.0,9.210315
3,0,3.0,9.028324
4,0,4.0,8.896585
5,0,5.0,9.603485
6,0,6.0,9.137472
7,0,7.0,9.528781
8,0,8.0,9.694731
9,0,9.0,8.966652


#### more orders in a wave - work intensity leads to more delays

In [ ]:
df_info_delivered_clean_adhoc.merge(df_wave_exp, on='order_id', how='left')\
.groupby('order_count')['is_ontime'].mean()

,is_ontime
order_count,
1,0.949438
2,0.897576
3,0.841568
4,0.818516
5,0.812663
6,0.804754
7,0.805308
8,0.811244
9,0.809404


##### avg. number of orders in the wave by hour

In [ ]:
df_info_delivered_clean_adhoc.merge(df_wave_exp, on='order_id', how='left').groupby('grab_hour')\
[['order_count']].mean().reset_index()

,grab_hour,order_count
0,0.0,2.836295
1,1.0,3.326869
2,2.0,3.253394
3,3.0,2.770492
4,4.0,2.227171
5,5.0,2.391876
6,6.0,2.581804
7,7.0,3.104785
8,8.0,3.576244
9,9.0,2.834636


#### peak hour traffic can lead to delay

In [ ]:
df_info_delivered_clean_adhoc.merge(df_wave_exp, on='order_id', how='left').query('order_count<=3')\
.groupby(['order_count','grab_hour'])\
[['is_ontime']].mean().reset_index()

,order_count,grab_hour,is_ontime
0,1,0.0,0.937853
1,1,1.0,0.965686
2,1,2.0,0.942346
3,1,3.0,0.946237
4,1,4.0,0.938202
...,...,...,...
67,3,19.0,0.835288
68,3,20.0,0.834846
69,3,21.0,0.823192
70,3,22.0,0.816183
